In [1]:
# pip install duckdb
# pip install jupysql
# pip install notebook
# pip install tabulate
# pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [3]:
# If on windows, this only works if you're running Jupyter as an admin
# pip install duckdb-engine

  Using cached duckdb_engine-0.13.1-py3-none-any.whl (47 kB)
  Using cached packaging-24.1-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 20.9
    Uninstalling packaging-20.9:
      Successfully uninstalled packaging-20.9
Note: you may need to restart the kernel to use updated packages.


In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
import duckdb
import sqlalchemy
from datetime import date
import json
import pathlib
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [2]:
# Set up SQL connection
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Change the billboard_observation_date!

In [3]:
current_date = date.today().strftime('%Y-%m-%d')
billboard_observation_date = '2024-09-05'

In [34]:
# Replace with your filepath. Forward slashes work for both Mac and Windows
%sql duckdb:///C:/Users/Admin/billboard/raw_db.duckdb

In [5]:
%sql USE raw_db;
%sql USE raw_db.billboard;

,Success


In [6]:
%sql SELECT * FROM raw_db.billboard.artist LIMIT 2;

,artist_id,artist_name,pct_female,last_update
0,1,Zach Bryan,0.0,2024-08-23
1,2,Morgan Wallen,0.0,2024-08-23


In [19]:
# Update this for this week's rankings
rankings = [
    'A Bar Song (Tipsy)'
    , 'I Had Some Help'
    , "Ain't No Love In Oklahoma"
    , 'Cowgirls'
    , 'Pink Skies'
    , 'Miles On It'
    , 'Lies Lies Lies'
    , 'Austin'
    , 'Pour Me A Drink'
    , 'I Am Not Okay'
    , 'Guy For That'
    , '28'
    , 'High Road'
    , 'Dirt Cheap'
    , 'Losers'
    , 'You Look Like You Love Me'
    , 'Whiskey Whiskey'
    , 'Beautiful As You'
    , 'Lonely Road'
    , 'Hang Tight Honey'
    , 'Chevrolet'
    , 'Wrong Ones'
    , "Wind Up Missin' You"
    , "Think I'm In Love With You"
    , 'Love You, Miss You, Mean It'
    , 'California Sober'
    , 'Nosedive'
    , "What Don't Belong To Me"
    , 'Am I Okay?'
    , 'American Nights'
    , "She's Somebody's Daughter (Reimagined)"
    , 'Love You Again'
    , 'Finer Things'
    , 'Young Love & Saturday Nights'
    , "Missin' You Like This"
    , 'Holy Smokes'
    , 'Yours'
    , 'Devil You Know'
    , 'Hide My Gun'
    , 'Liar' # Jelly Roll (reentry)
    , 'Sweet Dreams'
    , 'I Never Lie' # Zach Top (RE)
    , 'Relapse'
    , 'Have The Heart'
    , 'Worst Way'
    , "What He'll Never Have" # Dylan Scott (RE)
    , 'Goes Without Saying'
    , 'Wanna Be Loved'
    , 'This Is My Dirt' # Justin Moore (RE)
    , 'Cowboys Cry Too' # Kelsea Ballerini & Noah Kahan (RE)
]

In [20]:
%%sql df_song_labeled <<
SELECT
    ss.song_id
    , ss.song_name
    , art.artist_name
    , s2a.relationship_type
    , ss.untracked_weeks_on_chart
FROM song ss
LEFT JOIN song_to_artist s2a
    ON ss.song_id = s2a.song_id
LEFT JOIN artist art
    ON s2a.artist_id = art.artist_id
;

In [21]:
df_song_labeled.tail(20)

,song_id,song_name,artist_name,relationship_type,untracked_weeks_on_chart
102,17,Blackbiird,Tanner Adell,Other main artist,0.0
103,17,Blackbiird,Beyonce,Lead artist,0.0
104,50,Whiskey Whiskey,Moneybagg Yo,Lead artist,0.0
105,2,Last Night,None,None,51.0
106,3,Thinkin' Bout Me,None,None,46.0
107,4,Fast Car,None,None,43.0
108,5,White Horse,None,None,26.0
109,6,World On Fire,None,None,32.0
110,7,Need A Favor,None,None,51.0
111,8,Pretty Little Poison,None,None,32.0


In [22]:
existing_song_to_rank = {}
new_song_to_rank = {}
for song, rank_ix in zip(rankings, range(1,51)):
    print(df_song_labeled.loc[df_song_labeled['song_name'] == song].to_markdown())
    song_id = input("Please enter the song_id or 'New Song'\n")

    if song_id == '': # hit enter to take the most common song ID
        song_id = int(
            df_song_labeled.loc[
                df_song_labeled['song_name'] == song
            ]['song_id'].mode()
        )
        
    if str(song_id).lower() == 'new song':
        new_song_to_rank[song] = rank_ix
    else:
        existing_song_to_rank[int(song_id)] = rank_ix

|    |   song_id | song_name          | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:-------------------|:--------------|:--------------------|---------------------------:|
| 16 |        34 | A Bar Song (Tipsy) | Shaboozey     | Lead artist         |                          0 |
Please enter the song_id or 'New Song'

|    |   song_id | song_name       | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:----------------|:--------------|:--------------------|---------------------------:|
| 20 |        38 | I Had Some Help | Morgan Wallen | Other main artist   |                          0 |
| 76 |        38 | I Had Some Help | Post Malone   | Lead artist         |                          0 |
Please enter the song_id or 'New Song'

|    |   song_id | song_name                 | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:--------------------------|:--------------|:------

Please enter the song_id or 'New Song'

|    |   song_id | song_name   | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:------------|:--------------|:--------------------|---------------------------:|
| 55 |        76 | Wrong Ones  | Tim McGraw    | Featured artist     |                          0 |
| 86 |        76 | Wrong Ones  | Post Malone   | Lead artist         |                          0 |
Please enter the song_id or 'New Song'

|    |   song_id | song_name           | artist_name    | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:--------------------|:---------------|:--------------------|---------------------------:|
| 12 |        30 | Wind Up Missin' You | Tucker Wetmore | Lead artist         |                          0 |
Please enter the song_id or 'New Song'

|    |   song_id | song_name                  | artist_name     | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:-----------------

|    |   song_id | song_name   | artist_name    | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:------------|:---------------|:--------------------|---------------------------:|
| 30 |        49 | Relapse     | Warren Zeiders | Lead artist         |                          0 |
Please enter the song_id or 'New Song'

|    |   song_id | song_name      | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:---------------|:--------------|:--------------------|---------------------------:|
| 68 |        90 | Have The Heart | Dolly Parton  | Featured artist     |                          0 |
| 98 |        90 | Have The Heart | Post Malone   | Lead artist         |                          0 |
Please enter the song_id or 'New Song'

|    |   song_id | song_name   | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:------------|:--------------|:--------------------|---------------------------:|
| 17 

In [16]:
print(json.dumps(existing_song_to_rank, indent = 2))
print(json.dumps(new_song_to_rank, indent = 2))

{
  "34": 1,
  "38": 2,
  "40": 3,
  "32": 4,
  "43": 5,
  "37": 6,
  "58": 7,
  "27": 8,
  "52": 9,
  "48": 10,
  "64": 11,
  "57": 12,
  "47": 13,
  "21": 14,
  "77": 15,
  "53": 16,
  "50": 17,
  "39": 18,
  "65": 19,
  "42": 20,
  "25": 21,
  "76": 22,
  "30": 23,
  "33": 24,
  "78": 25,
  "81": 26,
  "79": 27,
  "62": 28,
  "56": 29,
  "44": 30,
  "23": 31,
  "80": 32,
  "29": 33,
  "84": 34,
  "26": 35,
  "82": 36,
  "36": 37,
  "85": 38,
  "67": 39,
  "41": 40,
  "69": 41,
  "49": 42,
  "90": 43,
  "35": 44,
  "31": 45,
  "83": 46,
  "45": 47,
  "66": 48,
  "55": 49
}
{}


## Add artists on any new songs that aren't in the artist table yet to its CSV

    , 'Wrong Ones' # Post Malone feat Tim McGraw
    , 'Losers' # Post Malone feat Jelly Roll
    , 'California Sober' # Post Malone feat Chris Stapleton
    , "What Don't Belong To Me" # Post Malone
    , 'Finer Things' # Post Malone feat Hank Williams
    , 'Nosedive' # Post Malone feat Lainey Wilson
    , 'Yours' # Post Malone
    , 'Goes Without Saying' # Post Malone feat Brad Paisley
    , "Missin' You Like This" # Post Malone feat Luke Combs
    , 'Hide My Gun' # Post Malone feat HARDY
    , "Devil I've Been" # Post Malone feat ERNEST
    , 'Never Love You Again' # Post Malone feat Sierra Ferrell
    , 'M-E-X-I-C-O' # Post Malone feat Billy Strings
    , 'Right About You' # Post Malone

In [176]:
df_artist = pd.read_csv('source_data/artist.csv')

In [177]:
new_song_artists = [
    'Post Malone' # 15
    , 'Tim McGraw' # new
    , 'Jelly Roll' # 5
    , 'Chris Stapleton' # 12
    , 'Hank Williams' # new
    , 'Lainey Wilson' # 11
    , 'Brad Paisley' # new
    , 'Luke Combs' # 6
    , 'HARDY' # new
    , 'ERNEST' # 37
    , 'Sierra Ferrell' # new
    , 'Billy Strings' # new
    , 'Dolly Parton' # new
]
df_artist[df_artist['artist_name'].isin(new_song_artists)]

,artist_id,artist_name,pct_female,last_update
4,5,Jelly Roll,0.0,2024-08-23
5,6,Luke Combs,0.0,2024-08-23
10,11,Lainey Wilson,1.0,2024-08-23
11,12,Chris Stapleton,0.0,2024-08-23
14,15,Post Malone,0.0,2024-08-23
36,37,ERNEST,0.0,2024-08-23
54,55,Tim McGraw,0.0,2024-08-29
55,56,Hank Williams,0.0,2024-08-29
56,57,Brad Paisley,0.0,2024-08-29
57,58,HARDY,0.0,2024-08-29


In [178]:
df_artist['artist_id'].max()

60

In [180]:
df_artist = pd.concat(
    [
        df_artist
        , pd.DataFrame(
            np.array([
                [55, 'Tim McGraw', 0, current_date]
                , [56, 'Hank Williams', 0, current_date]
                , [57, 'Brad Paisley', 0, current_date]
                , [58, 'HARDY', 0, current_date]
                , [59, 'Sierra Ferrell', 0, current_date]
                , [60, 'Billy Strings', 0, current_date]
                , [61, 'Dolly Parton', 1, current_date]
            ])
            , columns = ['artist_id', 'artist_name', 'pct_female', 'last_update']
        )
    ]
    , ignore_index = True
)

In [181]:
df_artist.to_csv('source_data/artist.csv', index=False)

## Add all new songs to the song table CSV

In [182]:
df_song_raw = pd.read_csv('source_data/song.csv')

In [183]:
max_song_id = df_song_raw['song_id'].max()
array_new_songs = []
for song_name in [
    'Wrong Ones' # Post Malone feat Tim McGraw
    , 'Losers' # Post Malone feat Jelly Roll
    , 'California Sober' # Post Malone feat Chris Stapleton
    , "What Don't Belong To Me" # Post Malone
    , 'Finer Things' # Post Malone feat Hank Williams
    , 'Nosedive' # Post Malone feat Lainey Wilson
    , 'Yours' # Post Malone
    , 'Goes Without Saying' # Post Malone feat Brad Paisley
    , "Missin' You Like This" # Post Malone feat Luke Combs
    , 'Hide My Gun' # Post Malone feat HARDY
    , "Devil I've Been" # Post Malone feat ERNEST
    , 'Never Love You Again' # Post Malone feat Sierra Ferrell
    , 'M-E-X-I-C-O' # Post Malone feat Billy Strings
    , 'Right About You' # Post Malone
    'Have The Heart' # Post Malone feat Dolly Parton
]:
    max_song_id += 1
    array_new_songs += [[max_song_id, song_name, 0, current_date]]

array_new_songs

[[90, 'Have The Heart', 0, '2024-08-30']]

In [184]:
df_song_raw = pd.concat(
    [
        df_song_raw
        , pd.DataFrame(
            np.array(array_new_songs)
            , columns = ['song_id', 'song_name', 'untracked_weeks_on_chart', 'last_update']
        )
    ]
    , ignore_index = True
)

In [186]:
# Update untracked_weeks_on_chart if needed
df_song_raw[-len(array_new_songs)-1:]

,song_id,song_name,untracked_weeks_on_chart,last_update
88,89,Right About You,0.0,2024-08-29
89,90,Have The Heart,0,2024-08-30


In [187]:
df_song_raw.to_csv('source_data/song.csv', index=False)

## Add all new songs to the song_to_artist table CSV

In [188]:
df_artist[df_artist['artist_name'].isin(new_song_artists)]

,artist_id,artist_name,pct_female,last_update
4,5,Jelly Roll,0.0,2024-08-23
5,6,Luke Combs,0.0,2024-08-23
10,11,Lainey Wilson,1.0,2024-08-23
11,12,Chris Stapleton,0.0,2024-08-23
14,15,Post Malone,0.0,2024-08-23
36,37,ERNEST,0.0,2024-08-23
54,55,Tim McGraw,0.0,2024-08-29
55,56,Hank Williams,0.0,2024-08-29
56,57,Brad Paisley,0.0,2024-08-29
57,58,HARDY,0.0,2024-08-29


In [97]:
df_s2a_raw = pd.read_csv('source_data/song_to_artist.csv').fillna('')

In [189]:
array_new_s2a = []
for s2a in [ # song_id, artist_id, relationship_conjunction, relationship_index
    [76, 15, '', '']
    , [76, 55, '', 1]
    , [77, 15, '', '']
    , [77, 5, '', 1]
    , [78, 15, '', '']
    , [78, 12, '', 1]
    , [79, 15, '', '']
    , [80, 15, '', '']
    , [80, 56, '', 1]
    , [81, 15, '', '']
    , [81, 11, '', 1]
    , [82, 15, '', '']
    , [83, 15, '', '']
    , [83, 57, '', 1]
    , [84, 15, '', '']
    , [84, 6, '', 1]
    , [85, 15, '', '']
    , [85, 58, '', 1]
    , [86, 15, '', '']
    , [86, 37, '', 1]
    , [87, 15, '', '']
    , [87, 59, '', 1]
    , [88, 15, '', '']
    , [88, 60, '', 1]
    , [89, 15, '', '']
    , [90, 15, '', '']
    , [90, 61, '', 1]
]:
    if s2a[3] == '': # Only lead artists don't have a relationship index
        reln = 'Lead artist'
    elif s2a[2] == '': # Featured artists don't have an explicit relationship conjunction
        reln = 'Featured artist' 
    else:
        reln = 'Other main artist'
        
    array_new_s2a += [[
        '-'.join([str(s2a[0]), reln, str(s2a[3])])
        , s2a[0]
        , s2a[1]
        , reln
        , s2a[2]
        , s2a[3]
        , current_date
    ]]
    
array_new_s2a

[['90-Lead artist-', 90, 15, 'Lead artist', '', '', '2024-08-30'],
 ['90-Featured artist-1', 90, 61, 'Featured artist', '', 1, '2024-08-30']]

In [190]:
df_s2a_raw = pd.concat(
    [
        df_s2a_raw
        , pd.DataFrame(
            np.array(array_new_s2a)
            , columns = [
                'song_to_artist_pk'
                , 'song_id'
                , 'artist_id'
                , 'relationship_type'
                , 'relationship_conjunction'
                , 'relationship_index'
                , 'last_update'
            ]
        )
    ]
    , ignore_index = True
)

In [191]:
df_s2a_raw.tail(1)

,song_to_artist_pk,song_id,artist_id,relationship_type,relationship_conjunction,relationship_index,last_update
129,90-Featured artist-1,90,61,Featured artist,,1,2024-08-30


In [192]:
df_s2a_raw.to_csv('source_data/song_to_artist.csv', index=False)

## Convert new_song_to_rank to the id_to_rankix format of existing_song_to_rank
Then stack the two dicts of songs and insert to CSV with more info like current_date for last_update

In [23]:
all_song_to_rank = existing_song_to_rank.copy()
for song_name, rank in new_song_to_rank.items():
    songs = list(df_song_raw[df_song_raw['song_name'] == song_name]['song_id'])
    
    if len(songs) == 1:
        song_id = songs[0]
    elif len(songs) > 1:
        song_id = int(input(f'{str(songs)}: Choose ID of song actually at rank {rank}\n'))
    else:
        raise Exception(f'The song at rank {rank} is missing from the song CSV')
    
    all_song_to_rank[int(song_id)] = rank

In [24]:
array_new_ranking = [
    [
        '-'.join([billboard_observation_date, str(rank)])
        , billboard_observation_date
        , rank
        , song_id
        , current_date
    ]
    for song_id, rank
    in all_song_to_rank.items()
]

array_new_ranking # Make sure it's 50 rows

[['2024-09-05-1', '2024-09-05', 1, 34, '2024-09-06'],
 ['2024-09-05-2', '2024-09-05', 2, 38, '2024-09-06'],
 ['2024-09-05-3', '2024-09-05', 3, 40, '2024-09-06'],
 ['2024-09-05-4', '2024-09-05', 4, 32, '2024-09-06'],
 ['2024-09-05-5', '2024-09-05', 5, 43, '2024-09-06'],
 ['2024-09-05-6', '2024-09-05', 6, 37, '2024-09-06'],
 ['2024-09-05-7', '2024-09-05', 7, 58, '2024-09-06'],
 ['2024-09-05-8', '2024-09-05', 8, 27, '2024-09-06'],
 ['2024-09-05-9', '2024-09-05', 9, 52, '2024-09-06'],
 ['2024-09-05-10', '2024-09-05', 10, 48, '2024-09-06'],
 ['2024-09-05-11', '2024-09-05', 11, 64, '2024-09-06'],
 ['2024-09-05-12', '2024-09-05', 12, 57, '2024-09-06'],
 ['2024-09-05-13', '2024-09-05', 13, 47, '2024-09-06'],
 ['2024-09-05-14', '2024-09-05', 14, 21, '2024-09-06'],
 ['2024-09-05-15', '2024-09-05', 15, 77, '2024-09-06'],
 ['2024-09-05-16', '2024-09-05', 16, 53, '2024-09-06'],
 ['2024-09-05-17', '2024-09-05', 17, 50, '2024-09-06'],
 ['2024-09-05-18', '2024-09-05', 18, 39, '2024-09-06'],
 ['2024-09

In [25]:
df_billboard_ranking_raw = pd.concat(
    [
        pd.read_csv('source_data/billboard_ranking.csv').fillna('')
        , pd.DataFrame(
            np.array(array_new_ranking)
            , columns = [
                'billboard_ranking_pk'
                , 'billboard_observation_date'
                , 'billboard_rank'
                , 'song_id'
                , 'last_update'
            ]
        )
    ]
    , ignore_index = True
)

In [26]:
df_billboard_ranking_raw.to_csv('source_data/billboard_ranking.csv', index=False)

## Push to duckdb

In [35]:
# TODO: instead of purging and rebuilding, use the raw data to run updates
%sql --close duckdb:///C:/Users/Admin/billboard/raw_db.duckdb --quiet
pathlib.Path('raw_db.duckdb').unlink(missing_ok = True)

TODO: integrate into jupyter using `%python`

In a command terminal
- `python table_configs/run_schema_configs.py billboard_schema.sql`
- `python table_configs/run_table_configs.py`
- `python main.py`

## Run the ranking SQL script

In [124]:
with duckdb.connect('raw_db.duckdb') as conn:
    with open('analysis/artist_ranking.sql') as sql_file:
        artist_ranking = conn.sql(sql_file.read())
    
    # TODO: Grab headers from the SQL too
    df_artist_ranking = pd.DataFrame(artist_ranking.fetchall())

In [125]:
df_artist_ranking

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,15,Post Malone,41,41,41,41,41,41,18,18,18,18,18,18
1,1,Zach Bryan,30,30,30,61,61,61,9,9,9,9,9,9
2,2,Morgan Wallen,10,15,20,70,75,80,2,3,4,2,3,4
3,5,Jelly Roll,8,13,20,8,13,25,2,3,5,2,3,5
4,16,Koe Wetzel,10,10,10,10,10,10,2,2,2,2,2,2
5,6,Luke Combs,8,8,15,43,43,50,2,2,4,2,2,4
6,11,Lainey Wilson,8,8,10,8,43,45,2,2,3,2,2,3
7,4,Megan Moroney,8,8,8,8,8,8,3,3,3,3,3,3
8,25,Shaboozey,7,7,7,7,7,7,2,2,2,2,2,2
9,8,Bailey Zimmerman,6,6,6,6,6,6,2,2,2,2,2,2


In [112]:
with duckdb.connect('raw_db.duckdb') as conn:
    print(conn.sql(
        """
        SELECT billboard_observation_date, COUNT(1) n
        FROM raw_db.billboard.billboard_ranking
        GROUP BY 1
        ORDER BY 1 ASC
        """
    ))

┌────────────────────────────┬───────┐
│ billboard_observation_date │   n   │
│            date            │ int64 │
├────────────────────────────┼───────┤
│ 2024-08-07                 │    50 │
│ 2024-08-13                 │    50 │
│ 2024-08-23                 │    50 │
│ 2024-08-27                 │    50 │
│ 2024-09-05                 │    50 │
└────────────────────────────┴───────┘

